Use the BS model to calculate expected losses of tranches of a CDX which is 1.25billioninsize.EL(i.e.meanofthedistribution)is100 million of the entire portfolio. Simply set S0=100. Volatility is 0.3, risk-free rate is 5% (i.e. 0.05), time horizon is 1 year (T = 1). Evaluate the expected losses of equity tranche (0~3%), mezzanine tranches (3~7, 7~10), senior tranche (10%~30%) and super-senior tranche (30%~100%). [Hint: equity tranche value is BS(K1) – BS(K0), mezzanine value is BS(K2) – BS(K1), etc.]

In [1]:
import numpy as np
from scipy.stats import norm
from math import sqrt, pi, log, e, erf

In [2]:
class EquityOption:     
    RiskfreeRate = 0.05
    Vol = 0.3       
    def __init__ (self, *args):
        self.CallFlag = int(args[0])
        self.Spot = float(args[1])
        self.Strike = float(args[2])
        self.Maturity = float(args[3])
        self.DividendYield =  float(args[4])
        
    def __imul__(self, stock_split):
        self.Spot = self.Spot * stock_split 
        self.Strike = self.Strike * stock_split
        return self
    
    @staticmethod 
    def N(x): 
        return (0.5*(1+ erf(x / sqrt(2.0))))
    def d1(self):
        return float((log(self.Spot/self.Strike) + (self.RiskfreeRate - self.DividendYield + (self.Vol**2) / 2  ) * self.Maturity) / (self.Vol * sqrt(self.Maturity)));
    def d2(self): 
        return float(self.d1() - self.Vol*sqrt(self.Maturity))
    def BS_Option(self, v=Vol):
        self.Vol = v #essential step
        dfq = e**(-self.DividendYield * self.Maturity)
        dfr = e**(-self.RiskfreeRate * self.Maturity)
        if self.CallFlag: 
            return self.Spot * dfq* self.N(self.d1()) - self.Strike*dfr * self.N(self.d2())
        else: 
            return self.Strike*dfr * self.N(-self.d2()) - self.Spot * dfq* self.N(-self.d1())

In [6]:
sigma = 0.3
S0 = 100
rf = 0.05
T = 1
d=0
CallFlag =1
K0 = 0
K1_E = 1250*0.03
K2_M = 1250*0.07
K3_M = 1250*0.1
K4_Senior = 1250*0.3
K5_Super_Senior= 1250*1

In [7]:
C0 = 100
C1 = EquityOption(CallFlag, S0, K1_E, T, d).BS_Option() 
C2 = EquityOption(CallFlag, S0, K2_M, T, d).BS_Option()
C3 = EquityOption(CallFlag, S0, K3_M, T, d).BS_Option()
C4 = EquityOption(CallFlag, S0, K4_Senior, T, d).BS_Option()
C5 = EquityOption(CallFlag, S0, K5_Super_Senior, T, d).BS_Option() 
print("C0: ", C0)
print("C1: ", C1)
print("C2: ", C2)
print("C3: ", C3)
print("C4: ", C4)
print("C5: ", C5)


C0:  100
C1:  64.33023180124518
C2:  21.2686986727699
C3:  5.691524187581631
C4:  0.00013519529706777517
C5:  2.7755575615628914e-14


In [8]:
print('The expected losses of equity tranche is (in millions)', C1-C0)
print('The expected losses of mezzanine tranche1 is (in millions)', C2-C1)
print('The expected losses of mezzanine  tranche2 is (in millions)', C3-C2)
print('The expected losses of senior  tranche is (in millions)', C4- C3)
print('The expected losses of super-senior tranche is (in millions)', C5-C4)


The expected losses of equity tranche is (in millions) -35.669768198754824
The expected losses of mezzanine tranche1 is (in millions) -43.061533128475276
The expected losses of mezzanine  tranche2 is (in millions) -15.57717448518827
The expected losses of senior  tranche is (in millions) -5.691388992284564
The expected losses of super-senior tranche is (in millions) -0.0001351952970400196
